# Constraint入门

@Author: Jian Huang

@email: jian.huang@xtalpi.com

Main reference:
- https://www.rosettacommons.org/demos/latest/tutorials/Constraints_Tutorial/Constraints
- https://zhuanlan.zhihu.com/p/58897635

@关键词 constraint，限制文件，限制惩罚类型和函数

### 一. Introduction


首先让我们考虑一个实际的应用场景：当我们在进行某一个蛋白质的构象优化的时候，我们通过某些实验数据表征了解到某两个残基在空间中需要保持足够近的距离（以此满足形成某些相互作用，或比如二硫键等），此时，我们应该如何在rosetta中考虑这种“额外的实验数据”呢？

答案就是利用rosetta中的Constraint，对体系施加限制。

当我们在具备一定实验数据基础或特定生物体系的先验条件后，简单使用rosetta中默认的打分函数不一定能很好地、针对性的捕获到这种“考量”。

Constraint（限制）是一种常用来对打分函数的修饰，以更加便捷考虑这些额外的实验信息。本质而言，constraint是一种对打分函数的附加项，基于实验数据基础，人为定义施加在原有打分函数上偏向势。

Rosetta中Constraint的工作原理分为三个步骤：
1. 计算出给定构象下的某些测量值（比如，某键长、键角、局域电荷值等等）；
2. 定义一个合适的惩罚函数，该惩罚函数将决定测量值的优劣，从而给与罚分。（比如，将最理想键长作为某二次函数的极小值点，任何其他的偏离该理想值的键长都将给予的罚分值，如下图）
3. 在上一步的罚分的基础上，乘以一个权重，并直接加到Rosetta的打分函数中

<center><img src="./img/quadratic_punishment.jpg" width="300" height="300" align=center /><center/>
    
我们可以预想，在原来的能量函数上添加了constraint后，构想优化的结果将会偏向我们设定的限制。因为一旦偏离限制越远，惩罚函数将会施加越多的能量，体系能量升高。

### 二、一个简单Constraints的例子

这里我们将使用ROSETTA中的constraint file（限制文件）。

限制文件中按照一定格式规定限制类型、限制对象、限制的惩罚函数形式。

    -> 限制文件的格式：参考 https://new.rosettacommons.org/docs/latest/rosetta_basics/file_types/constraint-file
    
    -> 作者已经预先在data文件目录下准备了constraint_file。

! under linux shell

cat constraint_file

AtomPair CA 20 CA 6 LINEAR_PENALTY 9.0 0 0 1.0

解释：
AtomPair CA 20 CA 6     # 告诉rosetta去测量哪两个原子的距离：20位置的α-C和6位置的α-C（Rosetta Numbering）
LINEAR_PENALTY 9.0 0 0 1.0     # 告诉rosetta如何将测量出来的距离转化为罚分 -- 惩罚函数的形式

这里使用了LINEAR_PENALTY的函数来处理罚分，后面跟随的四个数字代表这个函数的参数：

第一个参数9.0：设置该两个原子的距离最优（能量最低）是9.0 Angstrom；

第二个参数和第三个参数：指定平坦区（flat）的数值和范围；（稍后详细解释）

第四个参数：指定线性惩罚的斜率，这里是1。

平坦区值和范围的意思是，当两个原子的距离超过了9.0，即使已经偏移最优距离，但此时如果设定该距离在平坦区域内，全部使用该平坦区的值作为罚分。

如下图：
<center> <img src="./img/flat_region.png" width = "600" height = "200" align=center /> <center/>

In [15]:
# 在1ubq_clean.pdb的例子上施加原子对的限制

from pyrosetta import *
pyrosetta.init()

# 初始化REF2015的打分函数对象
my_scorefxn = create_score_function('ref2015')

# reweight the score I need
# 需要先将原子对的所属定义的score项权重设定为1
my_scorefxn.set_weight(pyrosetta.rosetta.core.scoring.atom_pair_constraint, 1.0) 

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python36.ubuntu 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r260 2020.28+release.8ecab77aa50 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1076598603 seed_offset=0 real_seed=1076598603
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1076598603 RG_type=mt19937


In [16]:
from pyrosetta.rosetta.protocols.constraint_movers import *
cst_set = ConstraintSetMover()
cst_set.add_constraints(False) # True=在原有限制基础上额外再添加限制，False= 从文件中读取并覆盖所有的限制。
cst_set.constraint_file('./data/constraint_file')

In [17]:
# load pose from 1ubq_clean.pdb
initial_pose = pose_from_pdb("./data/1ubq_clean.pdb")

pose = Pose()
pose.assign(initial_pose)
cst_set.apply(pose)

core.import_pose.import_pose: File './data/1ubq_clean.pdb' automatically determined to be of type PDB
core.scoring.constraints.ConstraintsIO: read constraints from ./data/constraint_file
core.scoring.constraints.ConstraintsIO: Read in 1 constraints


In [18]:
# 打印查看当前pose所有设定的constraint
print(pose.constraint_set())

ResiduePairConstraints: total: 20   plotting active...
ResiduePairConstraints (6,20)
AtomPairConstraint (2,20-2,6)
         r      func     dfunc dfunc_est
         2     7.000     0.000    -1.000
       2.5     6.500     0.000    -1.000
         3     6.000     0.000    -1.000
       3.5     5.500     0.000    -1.000
         4     5.000     0.000    -1.000
       4.5     4.500     0.000    -1.000
         5     4.000     0.000    -1.000
       5.5     3.500     0.000    -1.000
         6     3.000     0.000    -1.000
       6.5     2.500     0.000    -1.000
         7     2.000     0.000    -1.000
       7.5     1.500     0.000    -1.000
         8     1.000     0.000    -1.000
       8.5     0.500     0.000    -1.000
         9     0.000     0.000     0.000
       9.5     0.500     0.000     1.000
        10     1.000     0.000     1.000
      10.5     1.500     0.000     1.000
        11     2.000     0.000     1.000
      11.5     2.500     0.000     1.000
        12     3.000    

### 小结

从这个简单的例子中可以看到，使用“限制文件”对pose进行限制的方法理论上适用于所有的场景。因为只要我们按照rosetta的要求编写所有的限制规则于限制文件中，这种方法就可以最大化用户自由度，但是缺点是使用“限制文件”事先需要对限制文件的编写规则有一定的了解和把握。



### 三、关于限制文件的编写

由于使用限制文件可以适用大多数情况下定义的限制，这里多费一些笔墨介绍限制文件的编写。

限制文件又可细分为**几何限制文件**和**序列特征限制文件**。每种限制文件都由以下两个部分组成：
1. 限制类型或内容 （what's being measured）
2. 限制/惩罚函数 （how that measured value is transformed into a scoring bonus/penalty）

限制文件的一般格式为每一行定义一种限制：

Constraint_Type1 Constraint_Def1

Constraint_Type2 Constraint_Def2

...

Constraint_Type会定义哪一种类型的测量值需要被限制，例如距离、角度、二面角等，并且会包含定义该测量值的一系列原子、残基编号（一般使用Rosetta numbering，如果要使用PDB numbering需要使用编号+链号，无空格的形式，例如“30A”；此外不支持PDB中的insertion code）等。

Constraint_Def部分会定义限制/惩罚函数的形式，即随着pose中该测量值偏移定义的理想值，应该如何施加额外的能量得分。


#### 3.1 单一限制类型

单一限制类型仅限制单一的测量值。

1. Atompair

<code>AtomPair Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Func_Type Func_Def</code>

score term： atom_pair_constraint

在Atom1和Atom2之间设置距离限制。AtomPairConstraint 可以与PDB numbering兼容

举例：

AtomPair CA 20 CA 6 LINEAR_PENALTY 9.0 0 0 1.0

2. NamedAtomPair

<code>NamedAtomPair Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Func_Def</code>

score term: atom_pair_constraint

在Atom1和Atom2之间设置距离限制。NamedAtomPair中原子以各自的名字信息进行储存，而非数字编号。当pose中的数字编号会变化的时候，NamedAtomPair仍然能够准确给定义的原子对施加限制。这种方法会牺牲一定的运行效率。当我们知道原子编号不会改变的时候，应该使用AtomPair。

3. Angle

<code>Angle Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Atom3_Name Atom3_ResNum Func_Type Func_Def</code>

score term: angle_constraint

限制向量(Atom2_Atom1)和向量(Atom2_Atom3)之间的角度。角度以弧度表示。

4. NamedAngle

<code>NamedAngle Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Atom3_Name Atom3_ResNum Func_Type Func_Def</code>

score term: angle_constraint

限制向量(Atom2_Atom1)和向量(Atom2_Atom3)之间的角度。角度以弧度表示。对应于Angle中的“NamedAtomPair”，原理类似。

5. Dihedral

<code>Dihedral Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Atom3_Name Atom3_ResNum Atom4_Name Atom4_ResNum Func_Type Func_Def</code>

score term: dihedral_constraint

由四个原子构成的二面角的限制，rosetta中的dihedral范围为 -pi ~ +pi

6. DihedralPair

<code>DihedralPair Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Atom3_Name Atom3_ResNum Atom4_Name Atom4_ResNum Atom5_Name Atom5_ResNum Atom6_Name Atom6_ResNum Atom7_Name Atom7_ResNum Atom8_Name Atom8_ResNum Func_Type Func_Def</code>

score term: dihedral_constraint

限制两个二面角相同。

7. CoordinateConstraint

<code>CoordinateConstraint Atom1_Name Atom1_ResNum[Atom1_ChainID] Atom2_Name Atom2_ResNum[Atom2_ChainID] Atom1_target_X_coordinate Atom1_target_Y_coordinate Atom1_target_Z_coordinate Func_Type Func_Def</code>

score term: coordinate_constraint

用于限制atom1原子的坐标为设定的“靶标” X Y Z值，以atom2的坐标作为参照系，观察atom1是否发生了运动（rosetta以此判断是否需要rescore）。在ResNum后面可以接chainDD说明支持PDB numbering。

8. LocalCoordinateConstraint

<code>LocalCoordinateConstraint Atom1_Name Atom1_ResNum Atom2_Name Atom3_Name Atom4_Name Atom234_ResNum Atom1_target_X_coordinate Atom1_target_Y_coordinate Atom1_target_Z_coordinate Func_Type Func_Def</code>

score term: coordinate_constraint

用于限制atom1原子的坐标为设定的“靶标” X Y Z值，以atom2，3，4三个原子定义的坐标系作为参考系。兼容PDN numbering。

9. AmbiguousNMRDistance

<code>AmbiguousNMRDistance Atom1_Name Atom1_ResNum Atom2_Name Atom2_ResNum Func_Type Func_Def</code>

score term: atom_pair_constraint

atom1和atom2的距离限制。与Atompair的区别是这里的原子名会被特别处理 -- 遍历所有的同类模糊氢原子 （即化学环境或实验/旋转相同的氢原子）。

10. SiteConstraint

<code>SiteConstraint Atom1_Name Atom1_ResNum Opposing_chain Func_Type Func_Def</code>

score term: atom_pair_constraint

限制atom1与某定义的chain具有相互作用。SiteConstraint可以施加一系列的模糊原子对限制，评估该原子是否与某一条链或某一个区域大体上是否存在相互作用。
具体而言，若我们在atom1施加了这种限制，那么其αC与另一条定义的链的所有αC之间都存在距离限制关系。在每一对距离限制的值计算出来后，只有最低的那个能量得分值才会作为SiteConstraint的最终得分。

11. SiteConstraintResidues

<code>SiteConstraintResidues Atom1_ResNum Atom1_Name Res2 Res3 Func_Type Func_Def</code>

score term: atom_pair_constraint

限制atom1至少与res2 3之中一个存在相互作用。atom1 + 其残基号用于定义特定原子与res2 3的αC的相互作用的限制。

12. BigBin

<code>BigBin res_number bin_char sdev</code>

score term: dihedral_constraint

限制某残基的二面角处于的范围，以字母进行表示。

 ‘O’ 表示cis-顺式的omega角度 [-10, 10]
 
 ‘G’ 表示在 [-100, 100] 的phi和psi正值角度
 
 ‘E’ 表示在 [100, -90] 的phi和psi正值角度
 
 ‘A’ 表示在 [-50, 30] 的phi和psi负值角度
 
 ‘B’ 表示在 [100, 175] 的phi和psi负值角度

#### 3.2 限制/惩罚函数

限制文件中每一行最后，都以<code>Func_Type Func_Def</code>定义惩罚函数的形式和参数

这里介绍几种简单、容易理解的限制/惩罚函数类型，众多更复杂的函数形式请参照上文提到的官网链接。

1. HARMONIC

    参数 x0 sd

    $$ f(x)=((x-x0) / sd)^2 $$
    

2. FLAT_HARMONIC 

    参数 x0 sd tol
    
    $$ f(x)=((x-x0) / sd)^2 $$
    
    在x0-tol到x0+tol之间惩罚值为0，其余部分按照原函数处理
    
    
3. GAUSSIANFUNC 

    参数 mean sd tag WEIGHT weight
    
    $$ f(x)=-ln(\frac{1}{sd*\sqrt{2\pi}} * exp(-\frac{(x-mean)^2}{2sd^2})) $$
    
4. LINEAR_PENALTY 

    参数 x0 depth width slope
    
    $$ f(x)= depth ;  x0 - depth \leq x \leq x0 - depth $$
    
    $$ f(x)= depth + slope * ( abs (x-x0) - width);  width \le abs (x-x0) $$

### 习题

1. 在1ubq_clean.pdb的pose中使用HARMONIC限制某二面角（可自己选择）。（注意score需要实现设定对应weight为1的开放状态；难度：*）